In [1]:
from choice_model import ChoiceModel
model = ChoiceModel()
prev_period_file = 'new_graph/new_graph_0.pkl'
new_model = ChoiceModel(graph_path=prev_period_file) 
period = 1

# Overview of previous period

In [2]:
model.roll_back(period-1)
Old_PPI, Old_CVI, Old_DCI = model.evaluate()
PPI, CVI, DCI = new_model.evaluate()
print("Old_PPI:",Old_PPI," PPI: ", PPI)
print("Old_CVI:",Old_CVI," CVI: ", CVI)
print("Old_DCI:",Old_DCI,"DCI: ", DCI)

Old_PPI: 0.3242945775964644  PPI:  0.31012110726643594
Old_CVI: 3.30188679245283  CVI:  2.843137254901961
Old_DCI: 0.02857142857142857 DCI:  0.0


# Simulate Period

In [3]:
# get all nodes in certain period 
graph = model.get_period(period)
new_nodes = [n for n in graph.nodes if n not in list(new_model.graph.nodes)]
new_nodes = [n for n in new_nodes if graph.nodes[n]['type'] != 'Long-term Project']
actor_ndoes = [n for n in graph.nodes if graph.nodes[n]['type'] == 'Actors']
other_nodes = [n for n in graph.nodes if graph.nodes[n]['type'] != 'Actors']
print("period: ",period)
print("number of nodes: ",len(new_nodes))
print("number of actors: ", len(actor_ndoes))
print("number of other nodes: ",len(other_nodes))

period:  1
number of nodes:  75
number of actors:  71
number of other nodes:  41


In [4]:
test_graph = new_model.graph.copy()
## for a totally new environemnt create an empty graph
# import networkx as nx
# test_graph = nx.DiGraph()

In [5]:
test_graph = model.predict_links(test_graph, new_nodes, file_name=f"new_graph/new_graph_{period}.pkl")

  1%|▏         | 1/75 [00:33<41:43, 33.83s/it]

add edge 300 1029
add edge 578 1029
add edge 282 1029
add edge 522 1029
add edge 256 1029


  3%|▎         | 2/75 [01:06<40:31, 33.31s/it]

add edge 177 1030
add edge 300 1030
add edge 193 1030
add edge 578 1030
add edge 522 1030
add edge 256 1030
add edge 282 1030
add edge 118 1030


  4%|▍         | 3/75 [01:19<28:29, 23.75s/it]

...

add edge 522 497
add edge 282 497
add edge 300 497
add edge 256 497
add edge 559 497
add edge 720 497
add edge 809 497
add edge 719 497
add edge 702 497
add edge 232 497


 99%|█████████▊| 74/75 [28:04<00:17, 17.78s/it]

add edge 282 243
add edge 256 243
add edge 515 243
add edge 300 243
add edge 522 243


100%|██████████| 75/75 [28:17<00:00, 22.64s/it]

add edge 256 250
add edge 282 250
add edge 702 250
add edge 522 250
add edge 300 250


In [8]:
new_model.graph.has_edge(461,690)

True

# Re-predict the isolated nodes

In [6]:
issolated_nodes = new_model.get_isolated_nodes()
print(len(issolated_nodes))

0


### if len(issolated_nodes) > 0

In [40]:
test_graph = new_model.graph.copy()
test_graph = model.predict_links(test_graph, issolated_nodes,file_name=f"new_graph_{period}.pkl")

 50%|█████     | 1/2 [00:20<00:20, 20.41s/it]

add edge 835 522
add edge 835 282
add edge 835 300
add edge 835 300


100%|██████████| 2/2 [00:42<00:00, 21.16s/it]

add edge 724 256
add edge 724 282
add edge 724 515
add edge 724 300


In [41]:
new_model = ChoiceModel(graph_path=f"new_graph_{period}.pkl")

issolated_nodes = new_model.get_isolated_nodes()
print(len(issolated_nodes))

#if isolated nodes stil exist, remove them
new_model.remove_isolated_nodes()
new_model.visualize(file_name=f"new_graph_{period}.html")

0


# Evaluate

In [44]:
model.roll_back(period)
prev_model = ChoiceModel(graph_path=f"new_graph_{period-1}.pkl")
OLD_PPI, OLD_CVI, OLD_DCI = model.evaluate()
PREV_PPI, PREV_CVI, PREV_DCI = prev_model.evaluate()
NEXT_PPI, NEXT_CVI, NEXT_DCI = new_model.evaluate(test_graph)
print("period: ",period)
print("OLD_PPI:",OLD_PPI," OLD_CVI: ", OLD_CVI, "OLD_DCI: ", OLD_DCI)
print("PREV_PPI:",PREV_PPI," PREV_CVI: ", PREV_CVI, "PREV_DCI: ", PREV_DCI)
print("Old_DCI:",NEXT_PPI,"DCI: ",  NEXT_CVI, "NEX_DCI: ", NEXT_DCI)

period:  2
OLD_PPI: 0.34454789123651813  OLD_CVI:  4.083333333333333 OLD_DCI:  0.7346938775510204
PREV_PPI: 0.5884246031746032  PREV_CVI:  3.57 PREV_DCI:  0.5602240896358541
Old_DCI: 0.6083953634085213 DCI:  3.65 NEX_DCI:  0.5479452054794519
